## Scratchpad for paper revisions

In [49]:
%load_ext autoreload
%autoreload 2
import os, sys
root_path = os.path.realpath('../')
sys.path.append(root_path)

import torch
from pathlib import Path

from utils.data import make_dataset
from utils.nnet import get_device

from hebbcl.logger import MetricLogger
from hebbcl.model import Nnet
from hebbcl.trainer import Optimiser, train_model
from hebbcl.parameters import parser


In [50]:
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]
# vars(args)


In [51]:
# override defaults 
args.gating = "oja"
args.perform_hebb = True
args.centering = True

In [52]:
# create dataset 
dataset = make_dataset(args)

# instantiate logger, model and optimiser:
logger = MetricLogger(save_dir)
model = Nnet(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)

True


In [53]:

# train model
train_model(args, model, optimiser, dataset, logger)

step 0, loss: task a -0.0591, task b 0.0159 | acc: task a 0.5000, task b 0.5000
... n_a: 5 n_b: 3
step 50, loss: task a -0.9163, task b 0.0351 | acc: task a 1.0000, task b 0.5000
... n_a: 7 n_b: 3
step 100, loss: task a -1.7001, task b 0.0033 | acc: task a 1.0000, task b 0.5000
... n_a: 9 n_b: 3
step 150, loss: task a -2.2966, task b 0.0286 | acc: task a 1.0000, task b 0.4500
... n_a: 10 n_b: 2
step 200, loss: task a -2.7774, task b 0.0104 | acc: task a 1.0000, task b 0.5000
... n_a: 10 n_b: 4
step 250, loss: task a -3.3645, task b 0.0272 | acc: task a 1.0000, task b 0.4000
... n_a: 11 n_b: 4
step 300, loss: task a -4.0166, task b -0.0189 | acc: task a 1.0000, task b 0.5000
... n_a: 14 n_b: 4
step 350, loss: task a -4.6642, task b 0.0089 | acc: task a 1.0000, task b 0.5000
... n_a: 11 n_b: 5
step 400, loss: task a -5.5624, task b 0.0144 | acc: task a 1.0000, task b 0.5000
... n_a: 13 n_b: 5
step 450, loss: task a -6.7005, task b 0.0028 | acc: task a 1.0000, task b 0.5000
... n_a: 12 n_

In [40]:
dataset['x_train'].shape

(10000, 27)

In [45]:
args.centering = False
dataset = make_dataset(args)
dataset["x_train"][:,-2:]

array([[2., 0.],
       [2., 0.],
       [2., 0.],
       ...,
       [0., 2.],
       [0., 2.],
       [0., 2.]])

In [46]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
dataset["x_train"][:,-2:] = sc.fit_transform(dataset["x_train"][:,-2:])

In [48]:
dataset["x_train"][:10,:]

array([[ 1.65816255e-01,  2.09611387e-01,  1.21313764e-01,
         3.21449473e-02,  3.89962455e-03,  5.35261429e-01,
         6.76633846e-01,  3.91605627e-01,  1.03765161e-01,
         1.25881422e-02,  7.91065111e-01,  1.00000000e+00,
         5.78755599e-01,  1.53354967e-01,  1.86040682e-02,
         5.35261429e-01,  6.76633846e-01,  3.91605627e-01,
         1.03765161e-01,  1.25881422e-02,  1.65816255e-01,
         2.09611387e-01,  1.21313764e-01,  3.21449473e-02,
         3.89962455e-03,  1.00000000e+00, -1.00000000e+00],
       [ 3.46111355e-04,  2.85302627e-03,  1.07672086e-02,
         1.86040682e-02,  1.47170293e-02,  2.85302627e-03,
         2.35177459e-02,  8.87550456e-02,  1.53354967e-01,
         1.21313764e-01,  1.07672086e-02,  8.87550456e-02,
         3.34958043e-01,  5.78755599e-01,  4.57833362e-01,
         1.86040682e-02,  1.53354967e-01,  5.78755599e-01,
         1.00000000e+00,  7.91065111e-01,  1.47170293e-02,
         1.21313764e-01,  4.57833362e-01,  7.91065111e-